In [4]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D
from tensorflow.keras import Model


TensorFlow version: 2.12.0


In [7]:
class cnnModel(Model):
  def __init__(self):
    super(cnnModel, self).__init__()
    # Conv layer + maxpool
    self.conv1 = Conv2D(512, 5, activation='relu')
    self.maxpool1 = MaxPool2D(pool_size=(5, 5), strides=2)

    self.conv2 = Conv2D(512, 5, activation='relu')
    self.maxpool2 = MaxPool2D(pool_size=(5, 5), strides=2)

    self.conv3 = Conv2D(128, 5, activation='relu')
    self.maxpool3 = MaxPool2D(pool_size=(5, 5), strides=2)

    #Fully connected layer
    self.flatten = Flatten()
    self.d1 = Dense(256, activation='relu')
    self.d2 = Dense(6, activation='softmax')

  def call(self, x):
    x = self.conv1(x)
    x = self.maxpool1(x)

    x = self.conv2(x)
    x = self.maxpool2(x)

    x = self.conv3(x)
    x = self.maxpool3(x)

    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

In [8]:
model = cnnModel()

In [9]:
loss_object = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [10]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')

In [11]:
@tf.function
def train_step(audio, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(audio, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)


In [13]:
@tf.function
def test_step(audio, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(audio, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)


In [ ]:
EPOCHS = 5

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for audio, labels in train_ds:
    train_step(audio, labels)

  for test_audio, test_labels in test_ds:
    test_step(test_audio, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )
